In [1]:
%%capture --no-stdout
import os
from scipy.stats import ks_2samp

from utils import *
from preprocessing import *
from models import *
import warnings
warnings.filterwarnings('ignore')

matplotlib.style.use('ggplot')
%matplotlib inline

In [3]:
def clean_descr(x):
    '''
    valid output :
    Airframe, Avionics, Paint, Engine, APU, Interior Modification, 
    Engineering, Rework, Other Miscellaneous Charge, Structural Repair
    
    other values found (converted to NaN):
        '307',   'tfe731',
        'aircraft',      'aircraft maintenance',
        'amendment of stc st02976ny non lpv equip',
        'caac validation for avixaero landing gear stc',
        'collins dual ahrs cl600-2b16',
        'conformity and flight test for wo 40250 casa cn235 project oda2ny-0064',
        'customer assist',
        'falcon 2000', 'falcon 2000lxs',  'falcon 7x',
        'falcon 900', 'falcon 900dx',   'falcon 900ex',
        'gmm rii items',  'rii requirements',
        'modification', 'modifications',
        'netjet credits',  'netjets rii section',
        'oda', 'southwest psu (amendment)',
        'stc certification of gogo satcom',
        'stc laseref iv aml stc',
        'weight and balance computation']
    '''
    if type(x)!=str:
        x=str(x)
    x=x.lower().strip()
    
    if x in ['engine','engine #1','engine #2','engine #3','#2 engine','engine # 1', 'enigine #1', 'engnie #1']:
        return 'Engine'
        
    elif x in ['airframe','airrfame','ariframe', 'airfame']: 
        return 'Airframe'
        
    elif x=='paint':
        return 'Paint'
    
    elif x=='apu':
        return 'APU'
        
    elif x in ['avionics','avioinics']:
        return 'Avionics'
    
    elif x in ['interior modifications','interior modifcations','interior modification',
               'interior moficiations','interior modfications','interiors modifications','inteiror modifications']:
        return 'Interior Modification'
        
    elif x in ['engineering','engneering','engineeering','engineering items','miggitt isfd updates engineering po sc-03235',
               'oda engineering', 'engineering oda', 'engineering sc17-02520','engineering support for csq15-00058']:
        return 'Engineering'
        
    elif x in ['rework','rework item','rework-','.rework', 're-work']:
        return 'Rework'
        
    elif x in ['structural repair','structural repairs']:
        return 'Structural Repair'
        
    elif x in ['other miscellaneous charges','other miscellaneeous charges','othe miscellaneous charges',
               'other miscellanesous charges','miscellaneous charges','other miscellanious charges',
               'miscelaneous charges','other misc charges','other miscellaneous', 'other miscellaneous items']:
        return 'Other Miscellaneous Charge'
    
    else:
        return ''

# Description level

In [4]:
wil = pd.read_csv(get_data_path('wilmington'))
columns_of_interest = ['WORKORDERKEY', 'DESCRIPTION',
                       #labor
                       'TOTALLABORESTIMATEDCOST', 'TOTALLABORCOST', 
                       'TOTALLABORESTIMATEDREVENUE', 'TOTALLABORREVENUE',
                       #parts
                       'TOTALPARTSESTIMATEDCOST', 'TOTALPARTSCOST',
                       'TOTALPARTSESTIMATEDREVENUE', 'TOTALPARTSREVENUE',
                       #labor hours
                       'ESTIMATEDLABORCOSTHOURS', 'ACTUALLABORCOSTHOURS',
                       #'ESTIMATEDLABORREVENUEHOURS', 'ACTUALLABORREVENUEHOURS'
                       
                       ]

df_i = wil[columns_of_interest]
df_i['description']=df_i.DESCRIPTION.apply(clean_descr)
df_i.drop('DESCRIPTION', axis=1, inplace=True)
df_group = df_i.groupby(['WORKORDERKEY', 'description']).sum().reset_index(level=(1))

df_group['LABOR_MARGIN']=df_group.TOTALLABORREVENUE - df_group.TOTALLABORCOST
df_group['PARTS_MARGIN']=df_group.TOTALPARTSREVENUE - df_group.TOTALPARTSCOST

df_group['EST_LABOR_MARGIN']=df_group.TOTALLABORESTIMATEDREVENUE - df_group.TOTALLABORESTIMATEDCOST
df_group['EST_PARTS_MARGIN']=df_group.TOTALPARTSESTIMATEDREVENUE - df_group.TOTALPARTSESTIMATEDCOST

df_group['cost_error']=df_group.ESTIMATEDLABORCOSTHOURS - df_group.ACTUALLABORCOSTHOURS


total_count = df_group.groupby('description').LABOR_MARGIN.count().rename('Total number of WO')

neg_labor = df_group[df_group.LABOR_MARGIN<0].groupby('description').LABOR_MARGIN.count()
neg_parts = df_group[df_group.PARTS_MARGIN<0].groupby('description').PARTS_MARGIN.count()
neg_cost_err = df_group[df_group.cost_error<0].groupby('description').cost_error.count()

less_labor = df_group[df_group.EST_LABOR_MARGIN>df_group.LABOR_MARGIN].groupby('description').LABOR_MARGIN.count()
less_parts = df_group[df_group.EST_PARTS_MARGIN>df_group.PARTS_MARGIN].groupby('description').PARTS_MARGIN.count()

neg_labor_p = 100*(neg_labor/total_count).rename('Negative Labor Margin').round(3)
neg_parts_p = 100*(neg_parts/total_count).rename('Negative Parts Margin').round(3)
neg_cost_err_p = 100*(neg_cost_err/total_count).rename('Underestimated Cost Hours').round(3)

less_labor_p = 100*(less_labor/total_count).rename('Labor Margin less than expected').round(3)
less_parts_p = 100*(less_parts/total_count).rename('Parts Margin less than expected').round(3)

res_w = pd.concat([total_count, neg_labor_p, neg_parts_p, neg_cost_err_p,  less_labor_p,less_parts_p], axis=1).drop('').reset_index().fillna(0)

res_w.to_csv('wilmington_data_analysis.csv', index=False)

In [5]:
reno = pd.read_csv(get_data_path('reno'))
columns_of_interest = ['WORKORDERKEY', 'DESCRIPTION',
                       #labor
                       'TOTALLABORESTIMATEDCOST', 'TOTALLABORCOST', 
                       'TOTALLABORESTIMATEDREVENUE', 'TOTALLABORREVENUE',
                       #parts
                       'TOTALPARTSESTIMATEDCOST', 'TOTALPARTSCOST',
                       'TOTALPARTSESTIMATEDREVENUE', 'TOTALPARTSREVENUE',
                       #labor hours
                       'ESTIMATEDLABORCOSTHOURS', 'ACTUALLABORCOSTHOURS',
                       'ESTIMATEDLABORREVENUEHOURS', 'ACTUALLABORREVENUEHOURS'
                       
                       ]

df_i = reno[columns_of_interest]
df_i['description']=df_i.DESCRIPTION.apply(clean_descr)
df_i.drop('DESCRIPTION', axis=1, inplace=True)
df_group = df_i.groupby(['WORKORDERKEY', 'description']).sum().reset_index(level=(1))

df_group['LABOR_MARGIN']=df_group.TOTALLABORREVENUE - df_group.TOTALLABORCOST
df_group['PARTS_MARGIN']=df_group.TOTALPARTSREVENUE - df_group.TOTALPARTSCOST

df_group['EST_LABOR_MARGIN']=df_group.TOTALLABORESTIMATEDREVENUE - df_group.TOTALLABORESTIMATEDCOST
df_group['EST_PARTS_MARGIN']=df_group.TOTALPARTSESTIMATEDREVENUE - df_group.TOTALPARTSESTIMATEDCOST

df_group['cost_error']=df_group.ESTIMATEDLABORCOSTHOURS - df_group.ACTUALLABORCOSTHOURS


total_count = df_group.groupby('description').LABOR_MARGIN.count().rename('Total number of WO')

neg_labor = df_group[df_group.LABOR_MARGIN<0].groupby('description').LABOR_MARGIN.count()
neg_parts = df_group[df_group.PARTS_MARGIN<0].groupby('description').PARTS_MARGIN.count()
neg_cost_err = df_group[df_group.cost_error<0].groupby('description').cost_error.count()

less_labor = df_group[df_group.EST_LABOR_MARGIN>df_group.LABOR_MARGIN].groupby('description').LABOR_MARGIN.count()
less_parts = df_group[df_group.EST_PARTS_MARGIN>df_group.PARTS_MARGIN].groupby('description').PARTS_MARGIN.count()

neg_labor_p = 100*(neg_labor/total_count).rename('Negative Labor Margin').round(3)
neg_parts_p = 100*(neg_parts/total_count).rename('Negative Parts Margin').round(3)
neg_cost_err_p = 100*(neg_cost_err/total_count).rename('Underestimated Cost Hours').round(3)

less_labor_p = 100*(less_labor/total_count).rename('Labor Margin less than expected').round(3)
less_parts_p = 100*(less_parts/total_count).rename('Parts Margin less than expected').round(3)

res_w = pd.concat([total_count, neg_labor_p, neg_parts_p, neg_cost_err_p,  less_labor_p,less_parts_p], axis=1).drop('').reset_index().fillna(0)


res_w.to_csv('reno_data_analysis.csv', index=False)

In [6]:
lr = pd.read_csv(get_data_path('littlerock'))
columns_of_interest = ['WORKORDERKEY', 'DESCRIPTION',
                       #labor
                       'TOTALLABORESTIMATEDCOST', 'TOTALLABORCOST', 
                       'TOTALLABORESTIMATEDREVENUE', 'TOTALLABORREVENUE',
                       #parts
                       'TOTALPARTSESTIMATEDCOST', 'TOTALPARTSCOST',
                       'TOTALPARTSESTIMATEDREVENUE', 'TOTALPARTSREVENUE',
                       #labor hours
                       'ESTIMATEDLABORCOSTHOURS', 'ACTUALLABORCOSTHOURS',
                       'ESTIMATEDLABORREVENUEHOURS', 'ACTUALLABORREVENUEHOURS'
                       
                       ]

df_i = lr[columns_of_interest]
df_i['description']=df_i.DESCRIPTION.apply(clean_descr)
df_i.drop('DESCRIPTION', axis=1, inplace=True)
df_group = df_i.groupby(['WORKORDERKEY', 'description']).sum().reset_index(level=(1))

df_group['LABOR_MARGIN']=df_group.TOTALLABORREVENUE - df_group.TOTALLABORCOST
df_group['PARTS_MARGIN']=df_group.TOTALPARTSREVENUE - df_group.TOTALPARTSCOST

df_group['EST_LABOR_MARGIN']=df_group.TOTALLABORESTIMATEDREVENUE - df_group.TOTALLABORESTIMATEDCOST
df_group['EST_PARTS_MARGIN']=df_group.TOTALPARTSESTIMATEDREVENUE - df_group.TOTALPARTSESTIMATEDCOST

df_group['cost_error']=df_group.ESTIMATEDLABORCOSTHOURS - df_group.ACTUALLABORCOSTHOURS


total_count = df_group.groupby('description').LABOR_MARGIN.count().rename('Total number of WO')

neg_labor = df_group[df_group.LABOR_MARGIN<0].groupby('description').LABOR_MARGIN.count()
neg_parts = df_group[df_group.PARTS_MARGIN<0].groupby('description').PARTS_MARGIN.count()
neg_cost_err = df_group[df_group.cost_error<0].groupby('description').cost_error.count()

less_labor = df_group[df_group.EST_LABOR_MARGIN>df_group.LABOR_MARGIN].groupby('description').LABOR_MARGIN.count()
less_parts = df_group[df_group.EST_PARTS_MARGIN>df_group.PARTS_MARGIN].groupby('description').PARTS_MARGIN.count()

neg_labor_p = 100*(neg_labor/total_count).rename('Negative Labor Margin').round(3)
neg_parts_p = 100*(neg_parts/total_count).rename('Negative Parts Margin').round(3)
neg_cost_err_p = 100*(neg_cost_err/total_count).rename('Underestimated Cost Hours').round(3)

less_labor_p = 100*(less_labor/total_count).rename('Labor Margin less than expected').round(3)
less_parts_p = 100*(less_parts/total_count).rename('Parts Margin less than expected').round(3)

res_w = pd.concat([total_count, neg_labor_p, neg_parts_p, neg_cost_err_p,  less_labor_p,less_parts_p], axis=1).drop('').reset_index().fillna(0)


res_w.to_csv('littlerock_data_analysis.csv', index=False)

# item level

In [5]:
wil = pd.read_csv(get_data_path('wilmington'))
columns_of_interest = ['WORKORDERKEY', 'ITEMNUMBER',
                       'TOTALLABORESTIMATEDCOST', 'TOTALLABORESTIMATEDREVENUE',
                       'TOTALPARTSESTIMATEDCOST', 'TOTALPARTSESTIMATEDREVENUE',
                       'TOTALPARTSCOST','TOTALPARTSREVENUE','TOTALLABORREVENUE','TOTALLABORCOST'
                       ]
df_i = wil[columns_of_interest]
df_group = df_i.groupby(['WORKORDERKEY', 'ITEMNUMBER']).sum().reset_index(level=(1))
df_group['LABOR_MARGIN']=df_group.TOTALLABORREVENUE - df_group.TOTALLABORCOST
df_group['PARTS_MARGIN']=df_group.TOTALPARTSREVENUE - df_group.TOTALPARTSCOST

df_group['EST_LABOR_MARGIN']=df_group.TOTALLABORESTIMATEDREVENUE - df_group.TOTALLABORESTIMATEDCOST
df_group['EST_PARTS_MARGIN']=df_group.TOTALPARTSESTIMATEDREVENUE - df_group.TOTALPARTSESTIMATEDCOST

total_count = df_group.groupby('ITEMNUMBER').LABOR_MARGIN.count().rename('Total number of WO')
neg_labor = df_group[df_group.LABOR_MARGIN<0].groupby('ITEMNUMBER').LABOR_MARGIN.count()
neg_parts = df_group[df_group.PARTS_MARGIN<0].groupby('ITEMNUMBER').PARTS_MARGIN.count()

less_labor = df_group[df_group.EST_LABOR_MARGIN>df_group.LABOR_MARGIN].groupby('ITEMNUMBER').LABOR_MARGIN.count()
less_parts = df_group[df_group.EST_PARTS_MARGIN>df_group.PARTS_MARGIN].groupby('ITEMNUMBER').PARTS_MARGIN.count()

neg_labor_p = 100*(neg_labor/total_count).rename('Negative Labor Margin').round(3)
neg_parts_p = 100*(neg_parts/total_count).rename('Negative Parts Margin').round(3)

less_labor_p = 100*(less_labor/total_count).rename('Labor Margin less than expected').round(3)
less_parts_p = 100*(less_parts/total_count).rename('Parts Margin less than expected').round(3)

res_w = pd.concat([neg_labor_p, neg_parts_p,less_labor_p,less_parts_p], axis=1).reset_index()
res_w.to_csv('wilmington_data_analysis.csv', index=False)

In [6]:
reno = pd.read_csv(get_data_path('reno'))
columns_of_interest = ['WORKORDERKEY', 'ITEMNUMBER',
                       'TOTALLABORESTIMATEDCOST', 'TOTALLABORESTIMATEDREVENUE',
                       'TOTALPARTSESTIMATEDCOST', 'TOTALPARTSESTIMATEDREVENUE',
                       'TOTALPARTSCOST','TOTALPARTSREVENUE','TOTALLABORREVENUE','TOTALLABORCOST'
                       ]
df_i = reno[columns_of_interest]
df_group = df_i.groupby(['WORKORDERKEY', 'ITEMNUMBER']).sum().reset_index(level=(1))
df_group['LABOR_MARGIN']=df_group.TOTALLABORREVENUE - df_group.TOTALLABORCOST
df_group['PARTS_MARGIN']=df_group.TOTALPARTSREVENUE - df_group.TOTALPARTSCOST

df_group['EST_LABOR_MARGIN']=df_group.TOTALLABORESTIMATEDREVENUE - df_group.TOTALLABORESTIMATEDCOST
df_group['EST_PARTS_MARGIN']=df_group.TOTALPARTSESTIMATEDREVENUE - df_group.TOTALPARTSESTIMATEDCOST

total_count = df_group.groupby('ITEMNUMBER').LABOR_MARGIN.count().rename('Total number of WO')
neg_labor = df_group[df_group.LABOR_MARGIN<0].groupby('ITEMNUMBER').LABOR_MARGIN.count()
neg_parts = df_group[df_group.PARTS_MARGIN<0].groupby('ITEMNUMBER').PARTS_MARGIN.count()

less_labor = df_group[df_group.EST_LABOR_MARGIN>df_group.LABOR_MARGIN].groupby('ITEMNUMBER').LABOR_MARGIN.count()
less_parts = df_group[df_group.EST_PARTS_MARGIN>df_group.PARTS_MARGIN].groupby('ITEMNUMBER').PARTS_MARGIN.count()

neg_labor_p = 100*(neg_labor/total_count).rename('Negative Labor Margin').round(3)
neg_parts_p = 100*(neg_parts/total_count).rename('Negative Parts Margin').round(3)

less_labor_p = 100*(less_labor/total_count).rename('Labor Margin less than expected').round(3)
less_parts_p = 100*(less_parts/total_count).rename('Parts Margin less than expected').round(3)

res_r = pd.concat([neg_labor_p, neg_parts_p,less_labor_p,less_parts_p], axis=1).reset_index()
res_r.to_csv('reno_data_analysis.csv', index=False)

In [7]:
lirock = pd.read_csv(get_data_path('littlerock'))
columns_of_interest = ['WORKORDERKEY', 'ITEMNUMBER',
                       'TOTALLABORESTIMATEDCOST', 'TOTALLABORESTIMATEDREVENUE',
                       'TOTALPARTSESTIMATEDCOST', 'TOTALPARTSESTIMATEDREVENUE',
                       'TOTALPARTSCOST','TOTALPARTSREVENUE','TOTALLABORREVENUE','TOTALLABORCOST'
                       ]
df_i = lirock[columns_of_interest]
df_group = df_i.groupby(['WORKORDERKEY', 'ITEMNUMBER']).sum().reset_index(level=(1))
df_group['LABOR_MARGIN']=df_group.TOTALLABORREVENUE - df_group.TOTALLABORCOST
df_group['PARTS_MARGIN']=df_group.TOTALPARTSREVENUE - df_group.TOTALPARTSCOST

df_group['EST_LABOR_MARGIN']=df_group.TOTALLABORESTIMATEDREVENUE - df_group.TOTALLABORESTIMATEDCOST
df_group['EST_PARTS_MARGIN']=df_group.TOTALPARTSESTIMATEDREVENUE - df_group.TOTALPARTSESTIMATEDCOST

total_count = df_group.groupby('ITEMNUMBER').LABOR_MARGIN.count().rename('Total number of WO')
neg_labor = df_group[df_group.LABOR_MARGIN<0].groupby('ITEMNUMBER').LABOR_MARGIN.count()
neg_parts = df_group[df_group.PARTS_MARGIN<0].groupby('ITEMNUMBER').PARTS_MARGIN.count()

less_labor = df_group[df_group.EST_LABOR_MARGIN>df_group.LABOR_MARGIN].groupby('ITEMNUMBER').LABOR_MARGIN.count()
less_parts = df_group[df_group.EST_PARTS_MARGIN>df_group.PARTS_MARGIN].groupby('ITEMNUMBER').PARTS_MARGIN.count()

neg_labor_p = 100*(neg_labor/total_count).rename('Negative Labor Margin').round(3)
neg_parts_p = 100*(neg_parts/total_count).rename('Negative Parts Margin').round(3)

less_labor_p = 100*(less_labor/total_count).rename('Labor Margin less than expected').round(3)
less_parts_p = 100*(less_parts/total_count).rename('Parts Margin less than expected').round(3)

res_l = pd.concat([neg_labor_p, neg_parts_p,less_labor_p,less_parts_p], axis=1).reset_index()
res_l.to_csv('littlerock_data_analysis.csv', index=False)